In [6]:
import duckdb

years = range(2000, 2024)
urls = [f"'https://raw.githubusercontent.com/JeffSackmann/tennis_atp/refs/heads/master/atp_matches_{year}.csv'" for year in years]
concat_urls = ", ".join(urls)
stats = duckdb.query(f"select * from read_csv([{concat_urls}]) where w_ace is not null").df()

stats

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2000-301,Auckland,Hard,32,A,20000110,1,103163,1.0,None,...,55,39,29,17,4,7,11.0,1612.0,63.0,595.0
1,2000-301,Auckland,Hard,32,A,20000110,2,102607,NaN,Q,...,32,25,18,12,3,6,211.0,157.0,49.0,723.0
2,2000-301,Auckland,Hard,32,A,20000110,3,103252,NaN,None,...,33,20,7,8,7,11,48.0,726.0,59.0,649.0
3,2000-301,Auckland,Hard,32,A,20000110,4,103507,7.0,None,...,43,29,14,10,6,8,45.0,768.0,61.0,616.0
4,2000-301,Auckland,Hard,32,A,20000110,5,102103,NaN,Q,...,46,34,18,12,5,9,167.0,219.0,34.0,873.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65365,2018-0605,Tour Finals,Hard,8,F,20181112,299,104925,1.0,None,...,33,23,7,8,5,9,1.0,8045.0,6.0,4310.0
65366,2018-0605,Tour Finals,Hard,8,F,20181112,300,100644,3.0,None,...,34,25,8,10,2,6,5.0,5085.0,1.0,8045.0
65367,2018-M-DC-2018-WG-M-FRA-CRO-01,Davis Cup WG F: FRA vs CRO,Clay,4,D,20181123,1,106432,NaN,None,...,64,43,18,15,9,13,12.0,2480.0,40.0,1050.0
65368,2018-M-DC-2018-WG-M-FRA-CRO-01,Davis Cup WG F: FRA vs CRO,Clay,4,D,20181123,2,105227,NaN,None,...,57,39,18,15,1,4,7.0,4250.0,259.0,200.0


In [29]:
clutch_stats = duckdb.query(
"""
with winner_centered as (
    select 
        winner_id as player_id, 
        winner_name as player_name,

        w_svpt as service_points,
        w_1stWon + w_2ndWon as service_points_won,

        l_svpt as return_points,
        l_svpt - (l_1stWon + l_2ndWon) as return_points_won,

        w_bpFaced as break_points_faced, 
        w_bpSaved as break_points_saved, 

        l_bpFaced as break_opportunities, 
        l_bpFaced - l_bpSaved as break_opportunities_converted
    from stats 
),
loser_centered as (
    select 
        loser_id as player_id, 
        loser_name as player_name, 

        l_svpt as service_points,
        l_1stWon + l_2ndWon as service_points_won,

        w_svpt as return_points,
        w_svpt - (w_1stWon + w_2ndWon) as return_points_won,

        l_bpFaced as break_points_faced, 
        l_bpSaved as break_points_saved, 
        
        w_bpFaced as break_opportunities, 
        w_bpFaced - w_bpSaved as break_opportunities_converted
    from stats 
),
combined as (
    select * from winner_centered
    union all
    select * from loser_centered
), player_level_stats as (
    select
        player_name,

        sum(break_points_saved) / sum(break_points_faced) as "Break Point Saved",
        sum(service_points_won) / sum(service_points) as "Service Points Won",

        sum(break_opportunities_converted) / sum(break_opportunities) as "Break Opportunities Converted",
        sum(return_points_won) / sum(return_points) as "Return Points Won",
    from combined
    group by all
), total_stats as (
    select
        'All Players' as player_name,

        sum(break_points_saved) / sum(break_points_faced) as "Break Point Saved",
        sum(service_points_won) / sum(service_points) as "Service Points Won",

        sum(break_opportunities_converted) / sum(break_opportunities) as "Break Opportunities Converted",
        sum(return_points_won) / sum(return_points) as "Return Points Won",
    from combined
    group by all
), unpivot_big_3 as (
    select * 
    from player_level_stats 
    unpivot (value for stat in ("Break Point Saved", "Service Points Won", "Break Opportunities Converted", "Return Points Won"))
    where player_name in ('Roger Federer', 'Rafael Nadal', 'Novak Djokovic')
), unpivot_total as (
    select * 
    from total_stats
    unpivot (value for stat in ("Break Point Saved", "Service Points Won", "Break Opportunities Converted", "Return Points Won"))
), combine_levels as (
    select * from unpivot_big_3
    union all
    select * from unpivot_total
)
select 
    *,
    case stat when 'Break Point Saved' then 'Serves'
        when 'Service Points Won' then 'Serves'
        when 'Break Opportunities Converted' then 'Returns'
        when 'Return Points Won' then 'Returns'
    end as is_serve,
    case stat when 'Break Point Saved' then 'Break Points'
        when 'Service Points Won' then 'All Points'
        when 'Break Opportunities Converted' then 'Break Points'
        when 'Return Points Won' then 'All Points'
    end as is_break_point,
from combine_levels
"""
).df()
clutch_stats

,player_name,stat,value,is_serve,is_break_point
0,Novak Djokovic,Break Point Saved,0.655609,Serves,Break Points
1,Novak Djokovic,Service Points Won,0.675611,Serves,All Points
2,Novak Djokovic,Break Opportunities Converted,0.441904,Returns,Break Points
3,Novak Djokovic,Return Points Won,0.421099,Returns,All Points
4,Roger Federer,Break Point Saved,0.675550,Serves,Break Points
5,Roger Federer,Service Points Won,0.697212,Serves,All Points
6,Roger Federer,Break Opportunities Converted,0.411880,Returns,Break Points
7,Roger Federer,Return Points Won,0.397948,Returns,All Points
8,Rafael Nadal,Break Point Saved,0.661325,Serves,Break Points
9,Rafael Nadal,Service Points Won,0.673245,Serves,All Points


In [30]:
import altair as alt

base = alt.Chart(clutch_stats).encode(
    x=alt.X('value', scale=alt.Scale(zero=False), title=None),
    y=alt.Y('player_name', title=None),
    color=alt.Color('is_serve', title=None),
)
circles = base.mark_point().encode(
    shape=alt.Shape('is_break_point', title=None)
)
lines = base.mark_line().encode(
    detail=['player_name', 'is_serve']
)
circles + lines

alt.LayerChart(...)